<a href="https://colab.research.google.com/github/precicionanalytics/invoicing/blob/main/PAAS_Invoice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Generating invoice

In [1]:
!pip install python-docx
!pip install python-docx reportlab Pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 23.1 MB/s eta 0:00:00


###GENERATE CONFIG FILE

In [ ]:
config_content = """
INVOICE_PREFIX=PAAS-
LLC_NAME=Precision Analytics & Strategy LLC
LLC_ADDRESS=6505 Roundrock Trail, Plano, Tx 75023
LLC_EMAIL=contact@precision-analytics.com
LLC_PHONE=+1 (469) 569-3289
BILL_TO=['Manifest Legal Tech, Inc.', '333 Park Ave. S.', 'New York, NY 10010']
ACCOUNT_NAME=Precision Analytics & Strategy LLC
ACCOUNT_NUMBER=488131348758
ROUTING_NUMBER=111000025
SWIFT=PAASUS33XXX
invoiceFileName=PAASInvManifestmmddyyyy idx.pdf
docxInvoiceFileName=invoice_mmddyyyy_idx.docx
PRICE=2500.00
ITEM_DESCRIPTION=Monthly Advertising Fee
"""

with open('config.txt', 'w') as f:
    f.write(config_content)

print("config.txt generated successfully!")

config.txt generated successfully!


###PDF invoice

### Regenerating PDF Invoice

In [2]:
import ast
from datetime import date, timedelta, datetime
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.lib.units import inch
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.platypus import Table, TableStyle
from PIL import Image
import pytz # Import pytz for timezone handling

def read_config(filepath):
    """
    Reads a configuration file and parses it into a dictionary of key-value pairs.
    Assumes each line in the file is in the format KEY=VALUE.

    Args:
        filepath (str): The path to the configuration file.

    Returns:
        dict: A dictionary containing the key-value pairs from the configuration file.
    """
    config_data = {}
    try:
        with open(filepath, 'r') as f:
            for line in f:
                line = line.strip() # Remove leading/trailing whitespace
                if line and '=' in line: # Ensure line is not empty and contains '='
                    key, value = line.split('=', 1) # Split only on the first '='
                    config_data[key.strip()] = value.strip()
    except FileNotFoundError:
        print(f"Error: The file '{filepath}' was not found.")
    except Exception as e:
        print(f"An error occurred while reading the config file: {e}")
    return config_data

# --- Dynamic Value Preparation ---

# Call the read_config function with the path to the config.txt file
config = read_config('config.txt')

# Define the CST timezone
cst_timezone = pytz.timezone('America/Chicago')

# Get today's date in CST
today_cst = datetime.now(cst_timezone).date()

# Format today's date as MM/DD/YYYY
INVOICE_DATE = today_cst.strftime("%m/%d/%Y")

# Calculate the due date by adding 15 days to today's date in CST
due_date_cst = today_cst + timedelta(days=15)

# Format the calculated due date as MM/DD/YYYY
DUE_DATE = due_date_cst.strftime("%m/%d/%Y")

# Construct the INVOICE_NO string
invoice_date_formatted = today_cst.strftime("%Y%m%d")
# Use 'INVOICE_PREFIX' from config if present, otherwise default to 'INV-'
invoice_prefix = config.get('INVOICE_PREFIX', 'INV-')
INVOICE_NO = f"{invoice_prefix}{invoice_date_formatted}-01"

# Extract LLC_NAME, LLC_ADDRESS, LLC_EMAIL, and LLC_PHONE from the config dictionary and strip quotes
LLC_NAME = config.get('LLC_NAME').strip('"') if config.get('LLC_NAME') else None
LLC_ADDRESS = config.get('LLC_ADDRESS').strip('"') if config.get('LLC_ADDRESS') else None
LLC_EMAIL = config.get('LLC_EMAIL').strip('"') if config.get('LLC_EMAIL') else None
LLC_PHONE = config.get('LLC_PHONE').strip('"') if config.get('LLC_PHONE') else None

# Create a list named BILL_TO by parsing the string representation from config
bill_to_str = config.get('BILL_TO')
if bill_to_str:
    try:
        BILL_TO = ast.literal_eval(bill_to_str)
    except (ValueError, SyntaxError):
        BILL_TO = [bill_to_str] # Fallback if parsing fails
else:
    BILL_TO = []

# Create a dictionary named BANK_DETAILS using correct keys from the config dictionary
BANK_DETAILS = {
    "account_name": config.get('ACCOUNT_NAME'),
    "account_number": config.get('ACCOUNT_NUMBER'),
    "routing_number": config.get('ROUTING_NUMBER'),
    "swift": config.get('SWIFT')
}

# --- Invoice Generation Logic ---

# CONFIG
LOGO_PATH = "PaasSmallLogo.png"
invoice_file_name_template = config.get('invoiceFileName', 'invoice.pdf')
date_for_filename = INVOICE_DATE.replace('/', '')
invoice_index = INVOICE_NO.split('-')[-1]
OUTPUT_FILE = invoice_file_name_template.replace('mmddyyyy', date_for_filename).replace('idx', invoice_index)

LLC_INFO = f"{LLC_NAME}\n{LLC_ADDRESS}\n{LLC_EMAIL}\n{LLC_PHONE}"

price_from_config = float(config.get('PRICE', '0.00')) # Default to 0.00 if not found
item_description = config.get('ITEM_DESCRIPTION', 'Monthly Advertising Fee')

ITEMS = [
    {"desc": item_description, "qty": 1, "price": price_from_config}
]

c = canvas.Canvas(OUTPUT_FILE, pagesize=letter)
width, height = letter
margin = 40

# ------------------------------------------------------
# LOGO + INVOICE HEADER
# ------------------------------------------------------
try:
    im = Image.open(LOGO_PATH)
    aspect = im.height / im.width
    logo_w = 1.5 * inch
    logo_h = logo_w * aspect
    c.drawImage(LOGO_PATH, margin, height - margin - logo_h, width=logo_w, height=logo_h, mask='auto')
except:
    logo_h = 0
    c.drawString(margin, height - margin, "[Logo missing]")

c.setFont("Helvetica-Bold", 22)
c.drawRightString(width - margin, height - margin - 10, "INVOICE")

# Add LLC_INFO below INVOICE text (right-aligned)
c.setFont("Helvetica", 10) # Smaller font for details
llc_info_lines = LLC_INFO.split('\n')
current_y_for_llc_info = height - margin - 30 # Starting y-coordinate for the first line of LLC_INFO

for line in llc_info_lines:
    # drawRightString aligns the right end of the string at (x, y)
    c.drawRightString(width - margin, current_y_for_llc_info, line)
    current_y_for_llc_info -= 10 # Adjusted for tighter line spacing

# current_y_for_llc_info now holds the Y-coordinate *below* the last drawn LLC_INFO line.
# We want to place the 'BILL TO' box's top edge below this, with some spacing.
bill_to_top = current_y_for_llc_info - 72 # Increased gap to move down 3-4 more lines


# ------------------------------------------------------
# BILL TO BOX (REDUCED HEIGHT + TIGHTER SPACING)
# ------------------------------------------------------

bill_to_height = 65   # reduced from 80
c.setFillColor(colors.whitesmoke)
c.rect(margin, bill_to_top - bill_to_height, width - 2*margin, bill_to_height, fill=1, stroke=0)
c.setFillColor(colors.black)

t = c.beginText(margin + 10, bill_to_top - 15)
t.setFont("Helvetica-Bold", 10)
t.textLine("BILL TO")
t.setFont("Helvetica", 10)
for line in BILL_TO:
    t.textLine(line)
c.drawText(t)

# Invoice metadata (right side)
meta_y = bill_to_top - 10
meta_x = width - margin - 200
c.setFont("Helvetica", 10)
c.drawString(meta_x, meta_y,      f"Invoice Number    : {INVOICE_NO}")
c.drawString(meta_x, meta_y - 14, f"Invoice Date         : {INVOICE_DATE}")
c.drawString(meta_x, meta_y - 28, f"Invoice Due Date  : {DUE_DATE}")

# ------------------------------------------------------
# ITEMS TABLE (NOW CLOSER TO BILL TO)
# ------------------------------------------------------
table_data = [["Item", "Quantity", "Price", "Amount"]]
subtotal = 0

for it in ITEMS:
    amt = it["qty"] * it["price"]
    table_data.append([it["desc"], it["qty"], f"${it['price']:,.2f}", f"${amt:,.2f}"])
    subtotal += amt

# Bordered table style
style = TableStyle([
    ('GRID', (0,0), (-1,-1), 0.8, colors.grey),
    ('BACKGROUND', (0,0), (-1,0), colors.lightgrey),
    ('FONTNAME', (0,0), (-1,0), 'Helvetica-Bold'),
    ('ALIGN', (1,1), (-1,-1), 'RIGHT'),
    ('VALIGN', (0,0), (-1,-1), 'MIDDLE'),
])

tbl = Table(table_data, colWidths=[310, 60, 80, 80])
tbl.setStyle(style)
table_w, table_h = tbl.wrap(0,0)

# PLACE TABLE MUCH CLOSER (old gap 30 \u2192 new gap 10)
table_y = (bill_to_top - bill_to_height) - 10 - table_h
tbl.drawOn(c, margin, table_y)

# ------------------------------------------------------
# AMOUNT DUE BOX (SHIFTED DOWN SO NO OVERLAP)
# ------------------------------------------------------
amount_box_w = 150
amount_box_h = 45
amount_box_x = width - margin - amount_box_w
amount_box_y = table_y - 60   # was -20, now pushed down 40 more

c.setFillColor(colors.lightgrey)
c.rect(amount_box_x, amount_box_y, amount_box_w, amount_box_h, fill=1, stroke=0)

c.setFillColor(colors.black)
c.setFont("Helvetica-Bold", 14)
c.drawCentredString(amount_box_x + amount_box_w/2, amount_box_y + 15, f"${subtotal:,.2f}")

c.setFont("Helvetica", 9)
c.drawCentredString(amount_box_x + amount_box_w/2, amount_box_y + 30, "Amount Due")

# ------------------------------------------------------
# PAYMENT INSTRUCTIONS (NOW BELOW AMOUNT BOX)
# ------------------------------------------------------
pay_box_y = amount_box_y - 110 - 20

c.setStrokeColor(colors.red)
c.setFillColor(colors.whitesmoke) # Set fill color for the box
c.rect(margin, pay_box_y, width - 2*margin, 110, fill=1, stroke=1) # Set fill=1 to apply background color

c.setFillColor(colors.black) # Reset fill color for text
t = c.beginText(margin + 10, pay_box_y + 95)
t.setFont("Helvetica-Bold", 12)
t.textLine("Payment instructions:")
t.setFont("Helvetica", 11)
t.textLine(f"Please submit payment within 15 days. A 7% late fee applies to payments received after {DUE_DATE}.")
t.textLine("")
t.textLine(f"Account name: {BANK_DETAILS['account_name']}")
t.textLine(f"Account number: {BANK_DETAILS['account_number']}")
t.textLine(f"Routing number: {BANK_DETAILS['routing_number']}")
c.drawText(t)

c.save()
print(f"Saved: {OUTPUT_FILE}")

Saved: InvManifest12142025 01.pdf


### Regenerating DOCX Invoice

In [ ]:
from docx.oxml import OxmlElement
from docx.oxml.ns import qn

def set_cell_shading(cell, fill_color):
    """
    Apply background shading to a table cell.
    fill_color should be a hex string like 'D3D3D3' (no #).
    """
    tc = cell._tc
    tcPr = tc.get_or_add_tcPr()
    shd = OxmlElement('w:shd')
    shd.set(qn('w:val'), 'clear')
    shd.set(qn('w:color'), 'auto')
    shd.set(qn('w:fill'), fill_color)
    tcPr.append(shd)


In [ ]:
from docx import Document
from docx.shared import Inches, Pt, RGBColor
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.enum.table import WD_ALIGN_VERTICAL, WD_TABLE_ALIGNMENT
from datetime import date, timedelta, datetime
import ast
import pytz # Import pytz for timezone handling

# --- Helper function to read config --- (Copied for self-containment)
def read_config(filepath):
    """
    Reads a configuration file and parses it into a dictionary of key-value pairs.
    Assumes each line in the file is in the format KEY=VALUE.

    Args:
        filepath (str): The path to the configuration file.

    Returns:
        dict: A dictionary containing the key-value pairs from the configuration file.
    """
    config_data = {}
    try:
        with open(filepath, 'r') as f:
            for line in f:
                line = line.strip() # Remove leading/trailing whitespace
                if line and '=' in line: # Ensure line is not empty and contains '='
                    key, value = line.split('=', 1) # Split only on the first '='
                    config_data[key.strip()] = value.strip()
    except FileNotFoundError:
        print(f"Error: The file '{filepath}' was not found.")
    except Exception as e:
        print(f"An error occurred while reading the config file: {e}")
    return config_data

# --- Dynamic Value Preparation (using config.txt) ---

# Call the read_config function with the path to the config.txt file
config = read_config('config.txt')

# Define the CST timezone
cst_timezone = pytz.timezone('America/Chicago')

# Get today's date in CST
today_cst = datetime.now(cst_timezone).date()

# Format today's date as MM/DD/YYYY
INVOICE_DATE = today_cst.strftime("%m/%d/%Y")

# Calculate the due date by adding 15 days to today's date in CST
due_date_cst = today_cst + timedelta(days=15)

# Format the calculated due date as MM/DD/YYYY
DUE_DATE = due_date_cst.strftime("%m/%d/%Y")

# Construct the INVOICE_NO string
invoice_date_formatted = today_cst.strftime("%Y%m%d")
# Use 'INVOICE_PREFIX' from config if present, otherwise default to 'INV-'
invoice_prefix = config.get('INVOICE_PREFIX', 'INV-')
INVOICE_NO = f"{invoice_prefix}{invoice_date_formatted}-01"

# Extract LLC_NAME, LLC_ADDRESS, LLC_EMAIL, and LLC_PHONE from the config dictionary and strip quotes
LLC_NAME = config.get('LLC_NAME').strip('"') if config.get('LLC_NAME') else None
LLC_ADDRESS = config.get('LLC_ADDRESS').strip('"') if config.get('LLC_ADDRESS') else None
LLC_EMAIL = config.get('LLC_EMAIL').strip('"') if config.get('LLC_EMAIL') else None
LLC_PHONE = config.get('LLC_PHONE').strip('"') if config.get('LLC_PHONE') else None

# Create a list named BILL_TO by parsing the string representation from config
bill_to_str = config.get('BILL_TO')
if bill_to_str:
    try:
        BILL_TO = ast.literal_eval(bill_to_str)
    except (ValueError, SyntaxError):
        BILL_TO = [bill_to_str] # Fallback if parsing fails
else:
    BILL_TO = []

# Create a dictionary named BANK_DETAILS using correct keys from the config dictionary
BANK_DETAILS = {
    "account_name": config.get('ACCOUNT_NAME'),
    "account_number": config.get('ACCOUNT_NUMBER'),
    "routing_number": config.get('ROUTING_NUMBER'),
    "swift": config.get('SWIFT')
}

# --- Invoice Generation Logic ---

# CONFIGURATION
LOGO_PATH = "PaasSmallLogo.png"   # Update if needed
invoice_file_name_template_docx = config.get('docxInvoiceFileName', 'invoice.docx')
date_for_filename = INVOICE_DATE.replace('/', '')
invoice_index = INVOICE_NO.split('-')[-1]
OUTPUT_FILE = invoice_file_name_template_docx.replace('mmddyyyy', date_for_filename).replace('idx', invoice_index)

price_from_config = float(config.get('PRICE', '0.00')) # Default to 0.00 if not found
item_description = config.get('ITEM_DESCRIPTION', 'Monthly Advertising Fee')

ITEMS = [
    {"desc": item_description, "qty": 1, "price": price_from_config}
]

doc = Document()

# Set default font and size
style = doc.styles['Normal']
font = style.font
font.name = 'Arial'
font.size = Pt(10)

# Section for logo and INVOICE title
table = doc.add_table(rows=1, cols=2)
table.autofit = False
# Adjust column widths to better match PDF layout (approx 2.5 inch for logo, rest for title)
table.columns[0].width = Inches(2.5)
table.columns[1].width = Inches(3.5)

logo_cell = table.cell(0, 0)
title_cell = table.cell(0, 1)

# Add Logo (left aligned)
try:
    p = logo_cell.paragraphs[0]
    r = p.add_run()
    r.add_picture(LOGO_PATH, width=Inches(1.5))
    p.alignment = WD_ALIGN_PARAGRAPH.LEFT
except Exception as e:
    logo_cell.text = "[Logo missing]"
    print(f"Error loading logo: {e}")

# Add INVOICE title (right aligned)
p = title_cell.paragraphs[0]
p.text = "INVOICE"
p.style = doc.styles['Heading 1']
p.alignment = WD_ALIGN_PARAGRAPH.RIGHT

# Add LLC details below INVOICE title in the same cell, right-aligned
p_llc_name = title_cell.add_paragraph(LLC_NAME)
p_llc_name.alignment = WD_ALIGN_PARAGRAPH.RIGHT

p_llc_address = title_cell.add_paragraph(LLC_ADDRESS)
p_llc_address.alignment = WD_ALIGN_PARAGRAPH.RIGHT

p_llc_email = title_cell.add_paragraph(LLC_EMAIL)
p_llc_email.alignment = WD_ALIGN_PARAGRAPH.RIGHT

p_llc_phone = title_cell.add_paragraph(LLC_PHONE)
p_llc_phone.alignment = WD_ALIGN_PARAGRAPH.RIGHT

doc.add_paragraph("") # Spacer

# Bill To and Invoice Details - using a table to mimic side-by-side layout
detail_table = doc.add_table(rows=1, cols=2)
detail_table.autofit = False
detail_table.columns[0].width = Inches(3.0) # Bill To width
detail_table.columns[1].width = Inches(3.0) # Invoice Details width
detail_table.alignment = WD_TABLE_ALIGNMENT.CENTER

# Bill To Cell
bill_to_cell = detail_table.cell(0, 0)
bill_to_p = bill_to_cell.paragraphs[0]
bill_to_p.add_run("BILL TO").bold = True
for line in BILL_TO:
    bill_to_cell.add_paragraph(line)
bill_to_cell.vertical_alignment = WD_ALIGN_VERTICAL.TOP

# Invoice Details Cell
inv_details_cell = detail_table.cell(0, 1)
inv_details_p = inv_details_cell.paragraphs[0]
inv_details_p.alignment = WD_ALIGN_PARAGRAPH.LEFT # Left align within cell for readability
inv_details_p.add_run(f"Invoice Number: {INVOICE_NO}\n")
inv_details_p.add_run(f"Invoice Date: {INVOICE_DATE}\n")
inv_details_p.add_run(f"Invoice Due Date: {DUE_DATE}")
inv_details_cell.vertical_alignment = WD_ALIGN_VERTICAL.TOP

doc.add_paragraph("") # Spacer

# Items Table
items_table = doc.add_table(rows=len(ITEMS) + 1, cols=4)
items_table.style = 'Table Grid'

# Table Headers
hdr_cells = items_table.rows[0].cells
hdr_cells[0].text = "Item"
hdr_cells[1].text = "Quantity"
hdr_cells[2].text = "Price"
hdr_cells[3].text = "Amount"
for cell in hdr_cells:
    cell.paragraphs[0].runs[0].bold = True
    cell.paragraphs[0].alignment = WD_ALIGN_PARAGRAPH.CENTER
    # Apply shading to header cells (light grey)
    #cell.fill.solid()
    #cell.fill.fore_color.rgb = RGBColor(0xD3, 0xD3, 0xD3)

subtotal = 0
# Table Data
for i, item in enumerate(ITEMS):
    row_cells = items_table.rows[i + 1].cells
    amt = item["qty"] * item["price"]
    row_cells[0].text = item["desc"]
    row_cells[1].text = str(item["qty"])
    row_cells[2].text = f"${item['price']:,.2f}"
    row_cells[3].text = f"${amt:,.2f}"
    subtotal += amt

    # Align quantity, price, and amount to the right
    row_cells[1].paragraphs[0].alignment = WD_ALIGN_PARAGRAPH.RIGHT
    row_cells[2].paragraphs[0].alignment = WD_ALIGN_PARAGRAPH.RIGHT
    row_cells[3].paragraphs[0].alignment = WD_ALIGN_PARAGRAPH.RIGHT

doc.add_paragraph("") # Spacer

# Amount Due Box (mimic by a right-aligned table)
amount_due_table = doc.add_table(rows=2, cols=1)
amount_due_table.autofit = False
amount_due_table.columns[0].width = Inches(2.0)
amount_due_table.alignment = WD_TABLE_ALIGNMENT.RIGHT

# Apply shading and style for 'Amount Due' header
amount_due_table.cell(0, 0).text = "Amount Due"
amount_due_table.cell(0, 0).paragraphs[0].runs[0].bold = True
amount_due_table.cell(0, 0).paragraphs[0].alignment = WD_ALIGN_PARAGRAPH.CENTER
#amount_due_table.cell(0, 0).fill.solid()
#amount_due_table.cell(0, 0).fill.fore_color.rgb = RGBColor(0xD3, 0xD3, 0xD3) # lightgrey

# Apply shading and style for the actual amount
amount_due_table.cell(1, 0).text = f"${subtotal:,.2f}"
amount_due_table.cell(1, 0).paragraphs[0].runs[0].bold = True
amount_due_table.cell(1, 0).paragraphs[0].runs[0].font.size = Pt(14)
amount_due_table.cell(1, 0).paragraphs[0].alignment = WD_ALIGN_PARAGRAPH.CENTER
#amount_due_table.cell(1, 0).fill.solid()
#amount_due_table.cell(1, 0).fill.fore_color.rgb = RGBColor(0xD3, 0xD3, 0xD3) # lightgrey

doc.add_paragraph("") # Spacer

# Payment Instructions Box (mimic PDF's box)
payment_inst_table = doc.add_table(rows=1, cols=1)
payment_inst_table.autofit = False
payment_inst_table.columns[0].width = Inches(6.5) # Roughly page width minus margins
payment_inst_table.style = 'Table Grid' # Adds borders to the box

pay_cell = payment_inst_table.cell(0, 0)
#pay_cell.fill.solid()
#pay_cell.fill.fore_color.rgb = RGBColor(0xF5, 0xF5, 0xF5) # Whitesmoke fill

pay_p = pay_cell.paragraphs[0]
pay_p.add_run("Payment instructions:\n").bold = True
pay_p.add_run("Please submit payment within 15 days. A 7% late fee applies to payments received after " + DUE_DATE + ".\n\n")
pay_p.add_run(f"Account name: {BANK_DETAILS['account_name']}\n")
pay_p.add_run(f"Account number: {BANK_DETAILS['account_number']}\n")
pay_p.add_run(f"Routing number: {BANK_DETAILS['routing_number']}\n")
pay_p.add_run(f"SWIFT Code (if required): {BANK_DETAILS['swift']}\n")

doc.add_paragraph("")
doc.add_paragraph("Thank you for your business!")

# Save the document
doc.save(OUTPUT_FILE)
print(f"Generated: {OUTPUT_FILE}")

Generated: invoice_12092025_01.docx


# Task
Generate 11 PDF invoices for each month from January 2026 to November 2026. For each invoice, set the invoice date to the 8th of that month and calculate the corresponding due date. Ensure that each generated PDF invoice has a unique filename that incorporates its specific invoice date (month and year).

## Modify PDF Invoice Generation Loop

### Subtask:
Modify the existing PDF invoice generation script to iterate through each month from January 2026 to November 2026. For each iteration, set the invoice date to the 8th of that month and calculate the corresponding due date.


**Reasoning**:
The subtask requires modifying the existing PDF invoice generation script (`e9bf7486`) to iterate through months from January 2026 to November 2026, setting the invoice date to the 8th of each month, calculating the due date, and generating a unique PDF for each. I will modify the relevant section to include a loop and update the dynamic date calculations and file naming within it.



In [5]:
import ast
from datetime import date, timedelta, datetime
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.lib.units import inch
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.platypus import Table, TableStyle
from PIL import Image
import pytz # Import pytz for timezone handling

def read_config(filepath):
    """
    Reads a configuration file and parses it into a dictionary of key-value pairs.
    Assumes each line in the file is in the format KEY=VALUE.

    Args:
        filepath (str): The path to the configuration file.

    Returns:
        dict: A dictionary containing the key-value pairs from the configuration file.
    """
    config_data = {}
    try:
        with open(filepath, 'r') as f:
            for line in f:
                line = line.strip() # Remove leading/trailing whitespace
                if line and '=' in line: # Ensure line is not empty and contains '='
                    key, value = line.split('=', 1) # Split only on the first '='
                    config_data[key.strip()] = value.strip()
    except FileNotFoundError:
        print(f"Error: The file '{filepath}' was not found.")
    except Exception as e:
        print(f"An error occurred while reading the config file: {e}")
    return config_data

# --- Dynamic Value Preparation (outside the loop for constants) ---

# Call the read_config function with the path to the config.txt file
config = read_config('config.txt')

# Define the CST timezone
cst_timezone = pytz.timezone('America/Chicago')

# Extract LLC_NAME, LLC_ADDRESS, LLC_EMAIL, and LLC_PHONE from the config dictionary and strip quotes
LLC_NAME = config.get('LLC_NAME').strip('"') if config.get('LLC_NAME') else None
LLC_ADDRESS = config.get('LLC_ADDRESS').strip('"') if config.get('LLC_ADDRESS') else None
LLC_EMAIL = config.get('LLC_EMAIL').strip('"') if config.get('LLC_EMAIL') else None
LLC_PHONE = config.get('LLC_PHONE').strip('"') if config.get('LLC_PHONE') else None

# Create a list named BILL_TO by parsing the string representation from config
bill_to_str = config.get('BILL_TO')
if bill_to_str:
    try:
        BILL_TO = ast.literal_eval(bill_to_str)
    except (ValueError, SyntaxError):
        BILL_TO = [bill_to_str] # Fallback if parsing fails
else:
    BILL_TO = []

# Create a dictionary named BANK_DETAILS using correct keys from the config dictionary
BANK_DETAILS = {
    "account_name": config.get('ACCOUNT_NAME'),
    "account_number": config.get('ACCOUNT_NUMBER'),
    "routing_number": config.get('ROUTING_NUMBER'),
    "swift": config.get('SWIFT')
}

# CONFIG for items and logo path
LOGO_PATH = "PaasSmallLogo.png"
invoice_file_name_template = config.get('invoiceFileName', 'invoice.pdf')

price_from_config = float(config.get('PRICE', '0.00')) # Default to 0.00 if not found
item_description = config.get('ITEM_DESCRIPTION', 'Monthly Advertising Fee')

ITEMS = [
    {"desc": item_description, "qty": 1, "price": price_from_config}
]

# --- Loop for invoice generation ---

# Iterate from January 2026 to November 2026
for month_num in range(1, 12): # 1 for Jan, 11 for Nov
    year = 2026

    # Set invoice date to the 8th of the current month
    invoice_day = 8
    current_invoice_date = date(year, month_num, invoice_day)

    # Format today's date as MM/DD/YYYY
    INVOICE_DATE = current_invoice_date.strftime("%m/%d/%Y")

    # Calculate the due date by adding 15 days to today's date
    due_date = current_invoice_date + timedelta(days=15)

    # Format the calculated due date as MM/DD/YYYY
    DUE_DATE = due_date.strftime("%m/%d/%Y")

    # Construct the INVOICE_NO string
    invoice_date_formatted_no = current_invoice_date.strftime("%Y%m%d")
    # Use 'INVOICE_PREFIX' from config if present, otherwise default to 'INV-'
    invoice_prefix = config.get('INVOICE_PREFIX', 'INV-')
    INVOICE_NO = f"{invoice_prefix}{invoice_date_formatted_no}-01" # Assuming a fixed index for simplicity per month

    # Update OUTPUT_FILE to be unique for each invoice
    date_for_filename = current_invoice_date.strftime("%m%d%Y")
    invoice_index = INVOICE_NO.split('-')[-1]
    OUTPUT_FILE = invoice_file_name_template.replace('mmddyyyy', date_for_filename).replace('idx', invoice_index)

    LLC_INFO = f"{LLC_NAME}\n{LLC_ADDRESS}\n{LLC_EMAIL}\n{LLC_PHONE}"

    c = canvas.Canvas(OUTPUT_FILE, pagesize=letter)
    width, height = letter
    margin = 40

    # ------------------------------------------------------
    # LOGO + INVOICE HEADER
    # ------------------------------------------------------
    try:
        im = Image.open(LOGO_PATH)
        aspect = im.height / im.width
        logo_w = 1.5 * inch
        logo_h = logo_w * aspect
        c.drawImage(LOGO_PATH, margin, height - margin - logo_h, width=logo_w, height=logo_h, mask='auto')
    except:
        logo_h = 0
        c.drawString(margin, height - margin, "[Logo missing]")

    c.setFont("Helvetica-Bold", 22)
    c.drawRightString(width - margin, height - margin - 10, "INVOICE")

    # Add LLC_INFO below INVOICE text (right-aligned)
    c.setFont("Helvetica", 10) # Smaller font for details
    llc_info_lines = LLC_INFO.split('\n')
    current_y_for_llc_info = height - margin - 30 # Starting y-coordinate for the first line of LLC_INFO

    for line in llc_info_lines:
        # drawRightString aligns the right end of the string at (x, y)
        c.drawRightString(width - margin, current_y_for_llc_info, line)
        current_y_for_llc_info -= 10 # Adjusted for tighter line spacing

    # current_y_for_llc_info now holds the Y-coordinate *below* the last drawn LLC_INFO line.
    # We want to place the 'BILL TO' box's top edge below this, with some spacing.
    bill_to_top = current_y_for_llc_info - 72 # Increased gap to move down 3-4 more lines


    # ------------------------------------------------------
    # BILL TO BOX (REDUCED HEIGHT + TIGHTER SPACING)
    # ------------------------------------------------------

    bill_to_height = 65   # reduced from 80
    c.setFillColor(colors.whitesmoke)
    c.rect(margin, bill_to_top - bill_to_height, width - 2*margin, bill_to_height, fill=1, stroke=0)
    c.setFillColor(colors.black)

    t = c.beginText(margin + 10, bill_to_top - 15)
    t.setFont("Helvetica-Bold", 10)
    t.textLine("BILL TO")
    t.setFont("Helvetica", 10)
    for line in BILL_TO:
        t.textLine(line)
    c.drawText(t)

    # Invoice metadata (right side)
    meta_y = bill_to_top - 10
    meta_x = width - margin - 200
    c.setFont("Helvetica", 10)
    c.drawString(meta_x, meta_y,      f"Invoice Number    : {INVOICE_NO}")
    c.drawString(meta_x, meta_y - 14, f"Invoice Date         : {INVOICE_DATE}")
    c.drawString(meta_x, meta_y - 28, f"Invoice Due Date  : {DUE_DATE}")

    # ------------------------------------------------------
    # ITEMS TABLE (NOW CLOSER TO BILL TO)
    # ------------------------------------------------------
    table_data = [["Item", "Quantity", "Price", "Amount"]]
    subtotal = 0

    for it in ITEMS:
        amt = it["qty"] * it["price"]
        table_data.append([it["desc"], it["qty"], f"${it['price']:,.2f}", f"${amt:,.2f}"])
        subtotal += amt

    # Bordered table style
    style = TableStyle([
        ('GRID', (0,0), (-1,-1), 0.8, colors.grey),
        ('BACKGROUND', (0,0), (-1,0), colors.lightgrey),
        ('FONTNAME', (0,0), (-1,0), 'Helvetica-Bold'),
        ('ALIGN', (1,1), (-1,-1), 'RIGHT'),
        ('VALIGN', (0,0), (-1,-1), 'MIDDLE'),
    ])

    tbl = Table(table_data, colWidths=[310, 60, 80, 80])
    tbl.setStyle(style)
    table_w, table_h = tbl.wrap(0,0)

    # PLACE TABLE MUCH CLOSER (old gap 30 \u2192 new gap 10)
    table_y = (bill_to_top - bill_to_height) - 10 - table_h
    tbl.drawOn(c, margin, table_y)

    # ------------------------------------------------------
    # AMOUNT DUE BOX (SHIFTED DOWN SO NO OVERLAP)
    # ------------------------------------------------------
    amount_box_w = 150
    amount_box_h = 45
    amount_box_x = width - margin - amount_box_w
    amount_box_y = table_y - 60   # was -20, now pushed down 40 more

    c.setFillColor(colors.lightgrey)
    c.rect(amount_box_x, amount_box_y, amount_box_w, amount_box_h, fill=1, stroke=0)

    c.setFillColor(colors.black)
    c.setFont("Helvetica-Bold", 14)
    c.drawCentredString(amount_box_x + amount_box_w/2, amount_box_y + 15, f"${subtotal:,.2f}")

    c.setFont("Helvetica", 9)
    c.drawCentredString(amount_box_x + amount_box_w/2, amount_box_y + 30, "Amount Due")

    # ------------------------------------------------------
    # PAYMENT INSTRUCTIONS (NOW BELOW AMOUNT BOX)
    # ------------------------------------------------------
    pay_box_y = amount_box_y - 110 - 20

    c.setStrokeColor(colors.red)
    c.setFillColor(colors.whitesmoke) # Set fill color for the box
    c.rect(margin, pay_box_y, width - 2*margin, 110, fill=1, stroke=1) # Set fill=1 to apply background color

    c.setFillColor(colors.black) # Reset fill color for text
    t = c.beginText(margin + 10, pay_box_y + 95)
    t.setFont("Helvetica-Bold", 12)
    t.textLine("Payment instructions:")
    t.setFont("Helvetica", 11)
    t.textLine(f"Please submit payment within 15 days. A 7% late fee applies to payments received after {DUE_DATE}.")
    t.textLine("")
    t.textLine(f"Account name: {BANK_DETAILS['account_name']}")
    t.textLine(f"Account number: {BANK_DETAILS['account_number']}")
    t.textLine(f"Routing number: {BANK_DETAILS['routing_number']}")
    c.drawText(t)

    c.save()
    print(f"Saved: {OUTPUT_FILE}")


Saved: InvManifest01082026 01.pdf
Saved: InvManifest02082026 01.pdf
Saved: InvManifest03082026 01.pdf
Saved: InvManifest04082026 01.pdf
Saved: InvManifest05082026 01.pdf
Saved: InvManifest06082026 01.pdf
Saved: InvManifest07082026 01.pdf
Saved: InvManifest08082026 01.pdf
Saved: InvManifest09082026 01.pdf
Saved: InvManifest10082026 01.pdf
Saved: InvManifest11082026 01.pdf


## Final Task

### Subtask:
Confirm the successful generation of 11 PDF invoices, each with the correct invoice date and unique filename, as per the user's request.


## Summary:

### Q&A
Yes, the generation of 11 PDF invoices was successful. Each invoice has the correct invoice date (8th of the respective month) and a unique filename incorporating the month and year.

### Data Analysis Key Findings
*   **11 PDF Invoices Generated:** The script successfully created 11 distinct PDF invoices, covering each month from January 2026 to November 2026.
*   **Correct Invoice Dates:** Each generated invoice correctly used the 8th of its respective month as the invoice date.
*   **Unique and Formatted Filenames:** Every invoice received a unique filename that incorporated the specific invoice date (month and year). Examples include `InvManifest01082026 01.pdf` for January 2026 and `InvManifest11082026 01.pdf` for November 2026.
*   **Calculated Due Dates:** The due date for each invoice was accurately calculated by adding 15 days to the invoice date, as specified.

### Insights or Next Steps
*   The automated invoice generation process is robust and successfully handles date calculations and unique file naming, making it suitable for recurring billing cycles.
*   Consider implementing a verification step to programmatically open and check a sample of the generated PDFs for content accuracy beyond just dates and filenames.
